In [1]:
import socket
import spynnaker8 as p
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from numpy import genfromtxt

import sys
sys.path.insert(1, '../../miscelaneous')
from spinnorse_tools import fetch_files, plot_iho

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [2]:
dt = 1           # (ms) simulation timestep

cell_params = {'tau_m': 20.0,
               'tau_syn_E': 5.0,
               'tau_syn_I': 5.0,
               'v_rest': -65.0,
               'v_reset': -65.0,
               'v_thresh': -50.0,
               'tau_refrac': 0.0, # 0.1 originally
               'cm': 1,
               'i_offset': 0.0
               }

w = 1

In [3]:
#SpiNNaker Setup
delay = 2*dt
nb_steps = 200


ratio = 1 
nb_hl = 1
nb_nol = 1
nb_nhl = 1
nb_nil = 1

# Fetch available trains of spikes in path
path = "input/"
files = fetch_files(path)

# Stack Spike Trains
n = len(files)
spike_trains = np.zeros((nb_steps,n))
for i in range(n):

    spike_trains[:,i] = genfromtxt(files[i], delimiter=',') 
    i_spikes = spike_trains[:,i:i+1]
    i_indexes = np.asarray(np.where(i_spikes>0)).reshape(-1)

    
    node_id = p.setup(timestep=dt, min_delay=delay, max_delay=delay)     
    p.set_number_of_neurons_per_core(p.IF_curr_exp, 10) #  100 neurons per core   

    # Create Populations
    celltype = p.IF_curr_exp

    fcnn_i = p.Population(nb_nil,p.SpikeSourceArray(spike_times=(i_indexes)))
    fcnn_h = p.Population(nb_nhl, celltype(**cell_params), label="fcnn_h")
    fcnn_o = p.Population(nb_nol, celltype(**cell_params), label="fcnn_o")



    # Setup Connectivity
    cell_conn = p.AllToAllConnector()
    connections = {'il_hl': p.Projection(fcnn_i, fcnn_h, cell_conn,
                            receptor_type='excitatory',
                            synapse_type=p.StaticSynapse(weight=w, delay=0)),
                   'hl_ol': p.Projection(fcnn_h, fcnn_o, cell_conn,
                            receptor_type='excitatory',
                            synapse_type=p.StaticSynapse(weight=w, delay=0))}

    # Record Voltages and Spikes
    fcnn_i.record(["spikes"])
    fcnn_h.record(["v","spikes"])
    fcnn_o.record(["v","spikes"])


    # Start Simulation
    print("%d Running simulation..." % node_id)
    p.run(nb_steps)

    # Get recorded data
    z_il = fcnn_i.get_data("spikes")
    v_hl = fcnn_h.get_data("v")
    z_hl = fcnn_h.get_data("spikes")
    v_ol = fcnn_o.get_data("v")
    z_ol = fcnn_o.get_data("spikes")

    # End Simulation

    p.end()

    time.sleep(20)

    i_indexes = np.asarray(z_il.segments[0].spiketrains[0])
    i_spikes = np.zeros((nb_steps,nb_nil))
    for j in i_indexes.astype(int):
        i_spikes[j,:] = 1

    v_hl = np.array(v_hl.segments[0].filter(name="v")[0]).reshape((nb_steps,nb_nhl))
    o_indexes_hl = np.asarray(z_hl.segments[0].spiketrains[0])
    o_spikes_hl = np.zeros((nb_steps,nb_nhl))
    for j in o_indexes_hl.astype(int):
        o_spikes_hl[j,:] = 1

    v_ol = np.array(v_ol.segments[0].filter(name="v")[0]).reshape((nb_steps,nb_nol))
    o_indexes_ol = np.asarray(z_ol.segments[0].spiketrains[0])
    o_spikes_ol = np.zeros((nb_steps,nb_nol))
    for j in o_indexes_ol.astype(int):
        o_spikes_ol[j,:] = 1

    
    sfn = files[i].replace("input/","").replace(".csv","")
    
    np.savetxt("summary/spinnaker/" + "voltage_hl_" + sfn, v_hl, delimiter=",")
    np.savetxt("summary/spinnaker/" + "output_hl_" + sfn, o_spikes_hl, delimiter=",")    
    np.savetxt("summary/spinnaker/" + "voltage_ol_" + sfn, v_ol, delimiter=",")
    np.savetxt("summary/spinnaker/" + "output_ol_" + sfn, o_spikes_ol, delimiter=",")
        
    # Plot Stuff
    plot_iho(i_spikes, o_spikes_hl, v_hl, o_spikes_ol, v_ol, sfn, nb_steps, ratio)    


2020-12-26 15:50:59 INFO: Read cfg files: /home/jprb/myspin/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/jprb/myspin/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/jprb/.spynnaker.cfg
2020-12-26 15:50:59 INFO: Will search these locations for binaries: /home/jprb/myspin/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/jprb/myspin/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-12-26 15:50:59 WARNING: /home/jprb/SpiNNorse/experiments/pathway_1d/reports has 2 old reports that have not been closed
2020-12-26 15:50:59 WARNING: /home/jprb/SpiNNorse/experiments/pathway_1d/application_generated_data_files has 2 old reports that have not been closed
2020-12-26 15:50:59 INFO: Setting time scale factor to 1.
2020-12-26 15:50:59 INFO: Setting machine time step to 1000 micro-seconds.
2020-12-26 15:50:59 INFO: Simulating for 200 1.0ms timesteps using a hardware timestep of 1000us
2020-12-26 15:50:59 

['/home/jprb/myspin/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/jprb/myspin/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/jprb/.spynnaker.cfg']
0 Running simulation...


2020-12-26 15:50:59 INFO: Creating transceiver for 192.168.240.1
2020-12-26 15:50:59 INFO: Working out if machine is booted
2020-12-26 15:50:59 INFO: Found board with version [Version: SC&MP 3.2.5 at SpiNNaker:0:0:0 (built Thu Aug  1 10:15:06 2019)]
2020-12-26 15:50:59 INFO: Machine communication successful
2020-12-26 15:50:59 INFO: Detected a machine on IP address 192.168.240.1 which has 857 cores and 120.0 links
2020-12-26 15:50:59 INFO: Time 0:00:00.014014 taken by MachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-26 15:50:59 INFO: Time 0:00:00.029459 taken by PreAllocateResourcesForExtraMonitorSupport
2020-12-26 15:50:59 INFO: Time 0:00:00.000220 taken by NetworkSpecificationReport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-26 15:50:59 INFO: Ti

SpinnmanGenericProcessException: 
     Received exception class: SpinnmanInvalidParameterException 
     With message: Setting parameter SDRAM Allocation response base address to value 0 is invalid: Could not allocate 48 bytes of SDRAM 
     When sending to 0:0:0
     Stack trace: []
